In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression 

### Create Data

In [2]:
Npoints = 1000
(xstart,xstop) = (0,50)
x = np.linspace(xstart,xstop,Npoints).reshape(Npoints,1)
y = np.linspace(xstart,xstop,Npoints).reshape(Npoints,1)
xCoeff= np.random.randn(1)
yCoeff= np.random.randn(1)
const = np.random.randn(1)
print(f"Original Paramters (a,b,c) in y=ax+by+c : ({xCoeff},{yCoeff},{const})")
zTrue = xCoeff*x + yCoeff*y + const
noiseAmp = 10
bias = 0.5
zNoisyUpper = zTrue+noiseAmp*np.random.random(Npoints).reshape(Npoints,1)+bias
zNoisyLower = zTrue-noiseAmp*np.random.random(Npoints).reshape(Npoints,1)-bias
zNoisy = np.concatenate((zNoisyLower,zNoisyUpper),axis=0)
X = np.concatenate((x,x),axis=0)
Y = np.concatenate((y,y),axis=0)
z = np.concatenate((np.zeros((Npoints,1)),np.ones((Npoints,1))),axis=0)
X = np.concatenate((X,Y),axis = 1)
X = np.concatenate((X,z),axis = 1)
print(f"X.shape for the data: {X.shape}")
print(f"z.shape for the labels: {z.shape}")

Original Paramters (a,b,c) in y=ax+by+c : ([-0.05833125],[-0.26854955],[0.72453665])
X.shape for the data: (2000, 3)
z.shape for the labels: (2000, 1)


In [3]:
def sigma(z):
    '''Computes the sigmoid for a numpy array z
    '''        
    return 1/(1+np.exp(-z))

In [4]:
def computeGradient(X,y,z):
    '''Computes the gradient for the cross entropy loss function. X is an nxm numpy array of features, y is the (n,) numPy array of labels.
       z =  <theta,X> is an (n,) numPy array. Returns the (m+1,) gradient at the current iteration.
    '''
    (n,m) = X.shape      
    sigma_z = sigma(z)    
    estimation_error = sigma_z-y
    terms=[]
    for i in range(m+1):
        if i==0:
            v = 1
        else:
            v = X[:,i-1]
        
        currentterm = sum(v*sigma_z*(1-sigma_z)*estimation_error)
        terms.append(currentterm)        
        
    return (np.array(terms))/n
    
    

In [5]:
print(X.shape)
print(z.shape)
labels = z.reshape(2*Npoints)
print(labels.shape)
theta_est = np.array([const,xCoeff,yCoeff,1])
z_est = theta_est[0] + theta_est[1]*X[:,0]+theta_est[2]*X[:,1]+theta_est[3]*X[:,2]
print(z_est.shape)

(2000, 3)
(2000, 1)
(2000,)
(2000,)


In [6]:
def GradientDescent(X,y,start,step_size,tolerance,Niter):    
    ''' Will perform gradient descent on X (nxm), using target y (nx1) . No of parameters in problem = m + 1(bias)
    '''    
    gradNorm = 10000
    i = 0
    current_theta_estimate = start
    norms = []
    n = X.shape[0]            
    while i<Niter and gradNorm > tolerance:        
        z_estimate = current_theta_estimate[0]+current_theta_estimate[1]*X[:,0] + current_theta_estimate[2]*X[:,1] +  current_theta_estimate[3]*X[:,2]
        grad = computeGradient(X,y,z_estimate)
        gradNorm = np.linalg.norm(grad)
        norms.append(gradNorm)        
        current_theta_estimate = current_theta_estimate - step_size*grad
        i+=1
    plt.plot(np.array(list(range(1,i+1))),norms)
    plt.xlabel('Iterations')
    plt.ylabel('||gradient||')
    plt.grid(True)
    plt.title("Convergence of gradient descent")
    print(f"Started at location {start}")
    print(f"Finished with {i} iterations")
    print(f"Finished with gradient norm = {gradNorm}")
    print(f"Finished at location {current_theta_estimate}")
    return current_theta_estimate
    

In [13]:
def VanillaLogisticRegression(X,y):    
    '''Assumes that the input matrix is an nx2 numPy array with n observations and y is an nx1 target vector 
    '''
    (n,m) = X.shape
    theta_estimate_start = np.random.rand(m+1)
#     theta_estimate_start = np.array([-const,-xCoeff,-yCoeff,1])
    learning_rate = 1e-5
    err_tolerance = 1e-6
    MaxIter = 1000
    estimated_coeff = GradientDescent(X,y,theta_estimate_start,learning_rate,err_tolerance,MaxIter)
    return estimated_coeff

    
    
    

    
    

In [ ]:
z=z.reshape(2000,)
theta_est = VanillaLogisticRegression(X,z)
print(f"theta estimated [ theta0,theta1, theta2, theta3]: {theta_est}")
const_est = -theta_est[0]/theta_est[3]
xCoeff_est  = theta_est[1]/theta_est[3]
yCoeff_est = -theta_est[2]/theta_est[3]

# y_decision_boundary = m_est*X[:,0] -c_est
print(f"x coefficient estimated : {xCoeff_est}")
print(f"Original x coefficient : {xCoeff}")
print(f"y coefficient estimated : {yCoeff_est}")
print(f"Original y coefficient : {yCoeff}")
print(f"Intercept estimated : {const_est}")
print(f"Original intercept : {const}")
# y_pred_algo = list(map(lambda q: 0 if q<0 else 1,X[:,1]-y_decision_boundary))
# print(y_pred_algo[Npoints-5:Npoints+6])
# sumErr = sum(abs(y_pred_algo-y))
# print(f"Sum error: {sumErr}")
# errRate = sumErr/(2*Npoints)
# print(f"Error probability : {errRate}")
